## Chat Models - Sequential Chain


In [ ]:
import os
%pip install langchain
os.environ['OPENAI_API_KEY'] = 'API_KEY_HERE'

Once you've initiated a call to a language model, it's often useful to make subsequent calls. This method, where the output of one call serves as the input to the next, is especially valuable for more complex tasks.

In this notebook, we'll explore how to implement this using 'sequential chains'. These chains help you link multiple calls together, creating pipelines tailored for specific scenarios.


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.prompts import PromptTemplate

In [3]:
# Defining a chat model:
chat = ChatOpenAI(temperature=0.7)

In [8]:
# Brainstorming Chain:
template = """Act as a social marketer. You are responsible for generating a list of blog post ideas.
Here is some company context to help with your brainstorming: {context}
"""

system_message_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(template=template, input_variables=["context"]),
)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

# Create the chain:
brainstorming_chain = LLMChain(llm=chat, prompt=chat_prompt, output_key="ideas")

In [6]:
# Outline Chain:
template = """You are a content writer. Your task is to pick an idea from {ideas} and then write a blog post outline.
Here is some company context to help with your outline: {context}
"""
system_message_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(template=template, input_variables=["context", "ideas"]),
)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

outline_chain = LLMChain(llm=chat, prompt=chat_prompt, output_key="outline")

In [9]:
# Create the chain:
outline_chain = SequentialChain(
    chains=[brainstorming_chain, outline_chain],
    input_variables=["context"],
)

In [10]:
# Run the chain:
outline_chain.run(
    {
        "context": "Our company is a software company that builds products for developers.",
    }
)

'Title: The Top 10 Tools Every Developer Should Have in Their Toolkit\n\nI. Introduction\n    A. Importance of having the right tools for developers\n    B. Overview of the top 10 tools to be discussed\n\nII. Integrated Development Environment (IDE)\n    A. Definition and benefits of using an IDE\n    B. Examples of popular IDEs and their features\n    C. How an IDE enhances productivity and code quality\n\nIII. Version Control System (VCS)\n    A. Explanation of version control and its significance\n    B. Benefits of using a VCS in software development\n    C. Introduction to popular VCS tools like Git and SVN\n\nIV. Code Editor\n    A. Differentiating between IDE and code editor\n    B. Advantages of using a code editor for lightweight tasks\n    C. Examples of code editors and their key features\n\nV. Debugging and Testing Tools\n    A. Importance of debugging and testing in software development\n    B. Overview of popular debugging and testing tools\n    C. How these tools aid in 

## Increasing the chain by a length of 1

You could then increase the sequential chain by adding more chains.


In [38]:
import langchain

langchain.verbose = True

# Translation chain:
template = "Translate the article outline into {language}: {outline}"
prompt = PromptTemplate.from_template(template)
translation_chain = LLMChain(
    llm=chat, prompt=prompt, output_key="translation", verbose=True
)

# Create the a new SeqwentialChain with the translation chain added:
translated_article_outline_chain = SequentialChain(
    chains=[brainstorming_chain, outline_chain, translation_chain],
    input_variables=["context", "language"],
    verbose=True,
)

In [39]:
translated_article_outline_chain.run(
    {
        "context": "Our company is a software company that builds products for developers.",
        "language": "French",
    }
)



> Entering new SequentialChain chain...
